# Libraries

In [2]:
# imports
import pandas as pd
import numpy as np
import argparse

import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

# Import models from another notebook
%run model.ipynb
%run util.ipynb

# Preprocessing

### Load sudoku

In [3]:
# load inputs
sudokus = pd.read_csv('./datasets/sudoku-kaggle.csv').values
print("Full shape:", sudokus.shape)
# subset = sudokus.sample(n=50000).values
# print("Subset shape:", subset.shape)

Full shape: (1000000, 2)


In [162]:
# load inputs
sudokus = pd.read_csv('./datasets/sudokus-unl.csv').values
print("Full shape:", sudokus.shape)
# subset = sudokus.sample(n=50000).values
# print("Subset shape:", subset.shape)

Full shape: (49995, 2)


In [4]:
# Split in half
used, hold = train_test_split(sudokus, test_size=0.95)
print("Subset shape:", used.shape)

Subset shape: (50000, 2)


### Split into data and labels

In [5]:
# Split into puzzles and solutions
puzzles, solutions = used[:, 0], used[:, 1]
print("Number of puzzles:", len(puzzles))
print("Number of solutions:", len(solutions))

Number of puzzles: 50000
Number of solutions: 50000


In [163]:
# Split into puzzles and solutions
puzzles, solutions = sudokus[:, 0], sudokus[:, 1]
print("Number of puzzles:", len(puzzles))
print("Number of solutions:", len(solutions))

Number of puzzles: 49995
Number of solutions: 49995


### Reshape the data

In [6]:
# Reshape to 9x9
reshape_f = lambda x: np.reshape([int(digit) for digit in x], (9, 9))
reshaped_puzzles = np.array(list(map(reshape_f, puzzles)))
reshaped_solutions = np.array(list(map(reshape_f, solutions)))
print("Shape of puzzles:", reshaped_puzzles.shape)
print("Shape of solutions:", reshaped_solutions.shape)

Shape of puzzles: (50000, 9, 9)
Shape of solutions: (50000, 9, 9)


In [7]:
# To one-hot encoding
one_hot_puzzles = to_one_hot(reshaped_puzzles)
one_hot_solutions = to_one_hot(reshaped_solutions - 1, n_classes=9)
print("Shape of puzzles:", one_hot_puzzles.shape)
print("Shape of solutions:", one_hot_solutions.shape)

Shape of puzzles: (50000, 9, 9, 10)
Shape of solutions: (50000, 9, 9, 9)


### Train test split

In [8]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(one_hot_puzzles, one_hot_solutions, 
                                                    test_size=0.2, random_state=42)
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

Training data shape: (40000, 9, 9, 10)
Training labels shape: (40000, 9, 9, 9)
Testing data shape: (10000, 9, 9, 10)
Testing labels shape: (10000, 9, 9, 9)


In [9]:
# validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validating data shape:", X_val.shape)
print("Validating labels shape:", y_val.shape)

Training data shape: (32000, 9, 9, 10)
Training labels shape: (32000, 9, 9, 9)
Validating data shape: (8000, 9, 9, 10)
Validating labels shape: (8000, 9, 9, 9)


In [10]:
# Create new X_train from y_train
X_train_completed = to_puzzles(y_train)
X_val_completed = to_puzzles(y_val)
print("Training data shape:", X_train_completed.shape)
print("Training labels shape:", y_train.shape)
print("Validating data shape:", X_val_completed.shape)
print("Validating labels shape:", y_val.shape)

Training data shape: (32000, 9, 9, 10)
Training labels shape: (32000, 9, 9, 9)
Validating data shape: (8000, 9, 9, 10)
Validating labels shape: (8000, 9, 9, 9)


# Start training

In [58]:
# initialize model
model = get_model(X_train.shape[1:], model_id='dense_model')
print("Model input shape:", model.input.shape)
print("Model output shape:", len(model.output))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model input shape: (?, 9, 9, 10)
Model output shape: 81


In [64]:
%%time
early_stop = EarlyStopping(patience=2, verbose=1)

i = 1
for nb_epochs, nb_delete in zip(
#         [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round
#         [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]  # digit to pull off
    [10],
    [20]
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    model.fit(
        delete_digits(X_train_completed, nb_delete),  # delete digits from training sample
        [y_train[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(X_val_completed, nb_delete), # delete same amount of digit from validation sample
            [y_val[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=256,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )
    keras.models.save_model(model, "./models/unl-model-%d.h5" % nb_delete)

Pass n° 1 ...


KeyboardInterrupt: 

In [ ]:
# Split into puzzles and solutions
puzzles_t, solutions_t = hold[:, 0], hold[:, 1]
print("Number of puzzles:", len(puzzles_t))
print("Number of solutions:", len(solutions_t))

# Reshape to 9x9
reshaped_puzzles_t = np.array(list(map(reshape_f, puzzles_t)))
reshaped_solutions_t = np.array(list(map(reshape_f, solutions_t)))
print("Shape of puzzles:", reshaped_puzzles_t.shape)
print("Shape of solutions:", reshaped_solutions_t.shape)

# To one-hot encoding
one_hot_puzzles_t = to_one_hot(reshaped_puzzles_t)
one_hot_solutions_t = to_one_hot(reshaped_solutions_t - 1)
print("Shape of puzzles:", one_hot_puzzles_t.shape)
print("Shape of solutions:", one_hot_solutions_t.shape)

### Convolutional Layers

In [24]:
# initialize model
c_model = get_model(X_train.shape[1:], model_id='conv_model')
print("Model input shape:", c_model.input.shape)
print("Model output shape:", len(c_model.output))

Model input shape: (?, 9, 9, 10)
Model output shape: 81


In [27]:
%%time
# train model
c_model.fit(delete_digits(X_train_completed, 0), 
            [y_train[:, i, j, :] for i in range(9) for j in range(9)], 
            batch_size=128, 
            epochs=1, 
            verbose=1)
keras.models.save_model(c_model, "./models/conv_model-0.h5")

Epoch 1/1
40500/40500 [==============================] - 91s 2ms/step - loss: 48.2136 - dense_83_loss: 0.8554 - dense_84_loss: 0.6764 - dense_85_loss: 0.6912 - dense_86_loss: 0.7023 - dense_87_loss: 0.6883 - dense_88_loss: 0.6906 - dense_89_loss: 0.6770 - dense_90_loss: 0.6611 - dense_91_loss: 0.8915 - dense_92_loss: 0.6684 - dense_93_loss: 0.5002 - dense_94_loss: 0.5021 - dense_95_loss: 0.5245 - dense_96_loss: 0.5048 - dense_97_loss: 0.5083 - dense_98_loss: 0.5164 - dense_99_loss: 0.5074 - dense_100_loss: 0.6750 - dense_101_loss: 0.7086 - dense_102_loss: 0.5109 - dense_103_loss: 0.5216 - dense_104_loss: 0.5300 - dense_105_loss: 0.5334 - dense_106_loss: 0.5107 - dense_107_loss: 0.5209 - dense_108_loss: 0.5197 - dense_109_loss: 0.7111 - dense_110_loss: 0.6915 - dense_111_loss: 0.5157 - dense_112_loss: 0.5172 - dense_113_loss: 0.5300 - dense_114_loss: 0.5313 - dense_115_loss: 0.5182 - dense_116_loss: 0.5188 - dense_117_loss: 0.5142 - dense_118_loss: 0.7131 - dense_119_loss: 0.6877 - dens

In [35]:
%%time
early_stop = EarlyStopping(patience=2, verbose=1)

i = 1
for nb_epochs, nb_delete in zip(
        [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round
        [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]  # digit to pull off
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    c_model.fit(
        delete_digits(X_train_completed, nb_delete),  # delete digits from training sample
        [y_train[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(X_val_completed, nb_delete), # delete same amount of digit from validation sample
            [y_val[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=128,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )
    keras.models.save_model(c_model, "./models/conv_model-%d.h5" % nb_delete)

Pass n° 1 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 88s 2ms/step - loss: 6.1046 - dense_83_loss: 0.1267 - dense_84_loss: 0.0747 - dense_85_loss: 0.0917 - dense_86_loss: 0.0864 - dense_87_loss: 0.0821 - dense_88_loss: 0.0930 - dense_89_loss: 0.0881 - dense_90_loss: 0.0717 - dense_91_loss: 0.1311 - dense_92_loss: 0.0810 - dense_93_loss: 0.0538 - dense_94_loss: 0.0594 - dense_95_loss: 0.0661 - dense_96_loss: 0.0598 - dense_97_loss: 0.0657 - dense_98_loss: 0.0590 - dense_99_loss: 0.0544 - dense_100_loss: 0.0772 - dense_101_loss: 0.0913 - dense_102_loss: 0.0588 - dense_103_loss: 0.0708 - dense_104_loss: 0.0679 - dense_105_loss: 0.0665 - dense_106_loss: 0.0648 - dense_107_loss: 0.0719 - dense_108_loss: 0.0588 - dense_109_loss: 0.0892 - dense_110_loss: 0.0931 - dense_111_loss: 0.0612 - dense_112_loss: 0.0646 - dense_113_loss: 0.0678 - dense_114_loss: 0.0689 - dense_115_loss: 0.0720 - dense_116_loss: 0.0644 - dense_117_loss: 0

Pass n° 2 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 86s 2ms/step - loss: 6.3582 - dense_83_loss: 0.0857 - dense_84_loss: 0.0689 - dense_85_loss: 0.0808 - dense_86_loss: 0.0807 - dense_87_loss: 0.0810 - dense_88_loss: 0.0815 - dense_89_loss: 0.0799 - dense_90_loss: 0.0725 - dense_91_loss: 0.0873 - dense_92_loss: 0.0734 - dense_93_loss: 0.0740 - dense_94_loss: 0.0779 - dense_95_loss: 0.0714 - dense_96_loss: 0.0658 - dense_97_loss: 0.0691 - dense_98_loss: 0.0760 - dense_99_loss: 0.0675 - dense_100_loss: 0.0712 - dense_101_loss: 0.0825 - dense_102_loss: 0.0796 - dense_103_loss: 0.0801 - dense_104_loss: 0.0785 - dense_105_loss: 0.0803 - dense_106_loss: 0.0850 - dense_107_loss: 0.0814 - dense_108_loss: 0.0810 - dense_109_loss: 0.0848 - dense_110_loss: 0.0828 - dense_111_loss: 0.0804 - dense_112_loss: 0.0883 - dense_113_loss: 0.0811 - dense_114_loss: 0.0798 - dense_115_loss: 0.0844 - dense_116_loss: 0.0794 - dense_117_loss: 0

Epoch 2/2
40500/40500 [==============================] - 86s 2ms/step - loss: 4.7298 - dense_83_loss: 0.0592 - dense_84_loss: 0.0567 - dense_85_loss: 0.0579 - dense_86_loss: 0.0564 - dense_87_loss: 0.0610 - dense_88_loss: 0.0588 - dense_89_loss: 0.0627 - dense_90_loss: 0.0523 - dense_91_loss: 0.0631 - dense_92_loss: 0.0521 - dense_93_loss: 0.0517 - dense_94_loss: 0.0588 - dense_95_loss: 0.0567 - dense_96_loss: 0.0506 - dense_97_loss: 0.0550 - dense_98_loss: 0.0588 - dense_99_loss: 0.0543 - dense_100_loss: 0.0535 - dense_101_loss: 0.0566 - dense_102_loss: 0.0621 - dense_103_loss: 0.0593 - dense_104_loss: 0.0608 - dense_105_loss: 0.0584 - dense_106_loss: 0.0582 - dense_107_loss: 0.0622 - dense_108_loss: 0.0598 - dense_109_loss: 0.0664 - dense_110_loss: 0.0654 - dense_111_loss: 0.0585 - dense_112_loss: 0.0650 - dense_113_loss: 0.0578 - dense_114_loss: 0.0593 - dense_115_loss: 0.0607 - dense_116_loss: 0.0536 - dense_117_loss: 0.0555 - dense_118_loss: 0.0564 - dense_119_loss: 0.0583 - dense

Pass n° 3 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/3
40500/40500 [==============================] - 85s 2ms/step - loss: 6.0113 - dense_83_loss: 0.0623 - dense_84_loss: 0.0686 - dense_85_loss: 0.0725 - dense_86_loss: 0.0775 - dense_87_loss: 0.0694 - dense_88_loss: 0.0716 - dense_89_loss: 0.0772 - dense_90_loss: 0.0666 - dense_91_loss: 0.0661 - dense_92_loss: 0.0620 - dense_93_loss: 0.0734 - dense_94_loss: 0.0763 - dense_95_loss: 0.0776 - dense_96_loss: 0.0753 - dense_97_loss: 0.0777 - dense_98_loss: 0.0719 - dense_99_loss: 0.0734 - dense_100_loss: 0.0637 - dense_101_loss: 0.0741 - dense_102_loss: 0.0751 - dense_103_loss: 0.0822 - dense_104_loss: 0.0804 - dense_105_loss: 0.0785 - dense_106_loss: 0.0794 - dense_107_loss: 0.0823 - dense_108_loss: 0.0794 - dense_109_loss: 0.0718 - dense_110_loss: 0.0716 - dense_111_loss: 0.0786 - dense_112_loss: 0.0762 - dense_113_loss: 0.0773 - dense_114_loss: 0.0745 - dense_115_loss: 0.0740 - dense_116_loss: 0.0761 - dense_117_loss: 0

Epoch 2/3
40500/40500 [==============================] - 85s 2ms/step - loss: 4.2187 - dense_83_loss: 0.0491 - dense_84_loss: 0.0467 - dense_85_loss: 0.0501 - dense_86_loss: 0.0554 - dense_87_loss: 0.0488 - dense_88_loss: 0.0503 - dense_89_loss: 0.0502 - dense_90_loss: 0.0470 - dense_91_loss: 0.0508 - dense_92_loss: 0.0439 - dense_93_loss: 0.0484 - dense_94_loss: 0.0518 - dense_95_loss: 0.0523 - dense_96_loss: 0.0497 - dense_97_loss: 0.0533 - dense_98_loss: 0.0557 - dense_99_loss: 0.0495 - dense_100_loss: 0.0459 - dense_101_loss: 0.0520 - dense_102_loss: 0.0518 - dense_103_loss: 0.0560 - dense_104_loss: 0.0484 - dense_105_loss: 0.0561 - dense_106_loss: 0.0518 - dense_107_loss: 0.0574 - dense_108_loss: 0.0544 - dense_109_loss: 0.0489 - dense_110_loss: 0.0495 - dense_111_loss: 0.0514 - dense_112_loss: 0.0538 - dense_113_loss: 0.0493 - dense_114_loss: 0.0541 - dense_115_loss: 0.0562 - dense_116_loss: 0.0538 - dense_117_loss: 0.0542 - dense_118_loss: 0.0517 - dense_119_loss: 0.0505 - dense

...

Epoch 3/10
40500/40500 [==============================] - 83s 2ms/step - loss: 23.5828 - dense_83_loss: 0.2854 - dense_84_loss: 0.2902 - dense_85_loss: 0.2932 - dense_86_loss: 0.2962 - dense_87_loss: 0.2864 - dense_88_loss: 0.2919 - dense_89_loss: 0.2864 - dense_90_loss: 0.2851 - dense_91_loss: 0.2906 - dense_92_loss: 0.2872 - dense_93_loss: 0.2895 - dense_94_loss: 0.2903 - dense_95_loss: 0.2865 - dense_96_loss: 0.2864 - dense_97_loss: 0.2916 - dense_98_loss: 0.2940 - dense_99_loss: 0.2849 - dense_100_loss: 0.2859 - dense_101_loss: 0.2881 - dense_102_loss: 0.2880 - dense_103_loss: 0.2912 - dense_104_loss: 0.2922 - dense_105_loss: 0.2957 - dense_106_loss: 0.2909 - dense_107_loss: 0.2854 - dense_108_loss: 0.2938 - dense_109_loss: 0.2880 - dense_110_loss: 0.2857 - dense_111_loss: 0.2950 - dense_112_loss: 0.2900 - dense_113_loss: 0.2948 - dense_114_loss: 0.2949 - dense_115_loss: 0.2958 - dense_116_loss: 0.2902 - dense_117_loss: 0.2914 - dense_118_loss: 0.2887 - dense_119_loss: 0.2885 - den

Epoch 00003: early stopping
Pass n° 17 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/10
40500/40500 [==============================] - 82s 2ms/step - loss: 32.4503 - dense_83_loss: 0.3933 - dense_84_loss: 0.4000 - dense_85_loss: 0.4044 - dense_86_loss: 0.3972 - dense_87_loss: 0.4027 - dense_88_loss: 0.3983 - dense_89_loss: 0.3976 - dense_90_loss: 0.4018 - dense_91_loss: 0.3981 - dense_92_loss: 0.3954 - dense_93_loss: 0.4017 - dense_94_loss: 0.4029 - dense_95_loss: 0.3997 - dense_96_loss: 0.4002 - dense_97_loss: 0.4015 - dense_98_loss: 0.3950 - dense_99_loss: 0.3940 - dense_100_loss: 0.3995 - dense_101_loss: 0.4025 - dense_102_loss: 0.4010 - dense_103_loss: 0.3972 - dense_104_loss: 0.3997 - dense_105_loss: 0.4024 - dense_106_loss: 0.3967 - dense_107_loss: 0.3957 - dense_108_loss: 0.4009 - dense_109_loss: 0.3960 - dense_110_loss: 0.3979 - dense_111_loss: 0.4031 - dense_112_loss: 0.4023 - dense_113_loss: 0.4004 - dense_114_loss: 0.4038 - dense_115_loss: 0.4007 - dense_116_l

Epoch 2/10
40500/40500 [==============================] - 83s 2ms/step - loss: 29.6137 - dense_83_loss: 0.3597 - dense_84_loss: 0.3678 - dense_85_loss: 0.3688 - dense_86_loss: 0.3624 - dense_87_loss: 0.3638 - dense_88_loss: 0.3678 - dense_89_loss: 0.3605 - dense_90_loss: 0.3660 - dense_91_loss: 0.3636 - dense_92_loss: 0.3594 - dense_93_loss: 0.3663 - dense_94_loss: 0.3704 - dense_95_loss: 0.3645 - dense_96_loss: 0.3637 - dense_97_loss: 0.3657 - dense_98_loss: 0.3595 - dense_99_loss: 0.3618 - dense_100_loss: 0.3622 - dense_101_loss: 0.3674 - dense_102_loss: 0.3664 - dense_103_loss: 0.3617 - dense_104_loss: 0.3670 - dense_105_loss: 0.3688 - dense_106_loss: 0.3602 - dense_107_loss: 0.3619 - dense_108_loss: 0.3617 - dense_109_loss: 0.3614 - dense_110_loss: 0.3640 - dense_111_loss: 0.3667 - dense_112_loss: 0.3651 - dense_113_loss: 0.3658 - dense_114_loss: 0.3695 - dense_115_loss: 0.3678 - dense_116_loss: 0.3634 - dense_117_loss: 0.3657 - dense_118_loss: 0.3644 - dense_119_loss: 0.3671 - den

Epoch 3/10
40500/40500 [==============================] - 82s 2ms/step - loss: 28.4114 - dense_83_loss: 0.3445 - dense_84_loss: 0.3507 - dense_85_loss: 0.3579 - dense_86_loss: 0.3508 - dense_87_loss: 0.3527 - dense_88_loss: 0.3503 - dense_89_loss: 0.3443 - dense_90_loss: 0.3508 - dense_91_loss: 0.3510 - dense_92_loss: 0.3471 - dense_93_loss: 0.3549 - dense_94_loss: 0.3571 - dense_95_loss: 0.3492 - dense_96_loss: 0.3490 - dense_97_loss: 0.3514 - dense_98_loss: 0.3464 - dense_99_loss: 0.3470 - dense_100_loss: 0.3463 - dense_101_loss: 0.3546 - dense_102_loss: 0.3502 - dense_103_loss: 0.3479 - dense_104_loss: 0.3498 - dense_105_loss: 0.3551 - dense_106_loss: 0.3466 - dense_107_loss: 0.3480 - dense_108_loss: 0.3495 - dense_109_loss: 0.3472 - dense_110_loss: 0.3491 - dense_111_loss: 0.3526 - dense_112_loss: 0.3498 - dense_113_loss: 0.3501 - dense_114_loss: 0.3549 - dense_115_loss: 0.3489 - dense_116_loss: 0.3447 - dense_117_loss: 0.3502 - dense_118_loss: 0.3496 - dense_119_loss: 0.3499 - den

Epoch 00003: early stopping
Wall time: 1h 15min 4s


### Given 17 cells

In [131]:
given17 = pd.read_csv('./datasets/17-givens.csv').values
given17.shape

(49151, 1)

In [53]:
# Reshape to 9x9
given17 = np.array(list(map(reshape_f, given17[:, 0])))
print("Shape of puzzles:", given17.shape)

# To one-hot encoding
given17 = np.array(to_one_hot(given17))
print("Shape of puzzles:", given17.shape)

Shape of puzzles: (49151, 9, 9)
Shape of puzzles: (49151, 9, 9, 10)


### HardestSudokusThread

In [132]:
hst = pd.read_csv('./datasets/HardestSudokusThread.csv').values
hst.shape

(375, 1)

In [82]:
# Reshape to 9x9
hst = np.array(list(map(reshape_f, hst[:, 0])))
print("Shape of puzzles:", hst.shape)

# To one-hot encoding
hst = np.array(to_one_hot(hst))
print("Shape of puzzles:", hst.shape)

Shape of puzzles: (375, 9, 9)
Shape of puzzles: (375, 9, 9, 10)


### UNL Sudokus

In [90]:
import json

In [133]:
with open('./datasets/sudokus-unl.json') as read_file:
    sudokus_unl = json.load(read_file)

sudokus_unl = np.array(sudokus_unl)
sudokus_unl.shape

(472, 13)

In [94]:
sudokus_unl[0]

array(['99', 'Base', 'Omaha World-Herald', '1', '2', '5',
       'March 24, 2009', '', 'Puzzle added by Jason Gaare',
       '100000026200709010030060000000800700710526038003007000000010050050603007960000004',
       '1', 'GAC', '29'], dtype='<U81')

In [95]:
# Reshape to 9x9
sudokus_unl = np.array(list(map(reshape_f, sudokus_unl[:, 9])))
print("Shape of puzzles:", sudokus_unl.shape)

# To one-hot encoding
sudokus_unl = np.array(to_one_hot(sudokus_unl))
print("Shape of puzzles:", sudokus_unl.shape)

Shape of puzzles: (472, 9, 9)
Shape of puzzles: (472, 9, 9, 10)


In [109]:
results = np.expand_dims(np.array(deltas_t), axis=1)
results

array([[False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [

In [110]:
full_unl = np.concatenate((sudokus_unl, results), axis=1)
full_unl.shape

(472, 14)

In [127]:
solvable = full_unl[np.where(full_unl[:, -1] == 'True')]
np.where(full_unl[:, -1] == 'True')

(array([  1,   6,  12,  14,  28,  36,  48,  49,  50,  53,  55,  66,  73,
         76,  80,  84,  86,  87, 467], dtype=int64),)

In [128]:
# algorithm
solvable[:, -3]

array(['AC', 'GAC', 'GAC', 'GAC', 'SAC', 'AC', 'AC', 'AC', 'AC', 'AC',
       'AC', 'SAC', 'AC', 'Unsolved', 'AC', 'AC', 'AC', 'AC', 'GAC'],
      dtype='<U81')

In [130]:
# difficulty
solvable[:, 4]

array(['2', '1', '5', '5', '4', '3', '4', '1', '1', '1', '5', '4', '1',
       '1', '1', '1', '1', '1', '2'], dtype='<U81')

In [125]:
solvable[np.where(solvable[:, -3] == 'Unsolved')]

array([['161', 'Base', '', '1', '1', '5', '2-Sol', '2-Sol', '',
        '906070403000400200070023010500000100040208060003000005030700050007005000405010708',
        '2', 'Unsolved', '29', 'True']], dtype='<U81')

### Prepare new dataset

In [139]:
unl = np.concatenate((given17, hst, sudokus_unl), axis=0)
unl.shape

(49998, 1)

In [145]:
unl_sols = solve(unl)
unl_sols.shape

(49998, 1)

In [155]:
unl_dataset = np.concatenate((unl, unl_sols), axis=1)
unl_dataset.shape

(49998, 2)

In [159]:
unl_dataset = unl_dataset[unl_dataset[:, 1] != None]
unl_dataset.shape

(49995, 2)

In [161]:
write_csv(['puzzle', 'solution'], unl_dataset, "./datasets/sudokus-unl.csv")